In [ ]:
# Install required libraries
https://github.com/ZakriaJanjua/NLP-Project.git

# Import libraries
import pandas as pd
import os
from langdetect import detect
from transformers import AutoTokenizer, AutoModelForSequenceClassification, Trainer, TrainingArguments
from sklearn.metrics import f1_score


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 981.5/981.5 kB 20.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for langdetect: filename=langdetect-1.0.9-py3-none-any.whl size=993222 sha256=257a5fe755eab65ae3d60b8b7c355833c364d6b10182e9139c4ea27e60f8df49
  Stored in directory: /root/.cache/pip/wheels/95/03/7d/59ea870c70ce4e5a370638b5462a7711ab78fba2f655d05106
Successfully built langdetect


In [5]:
from google.colab import files
uploaded = files.upload()


Saving dev.zip to dev.zip


In [6]:
from google.colab import files
uploaded = files.upload()

Saving gold_labels.zip to gold_labels.zip


In [7]:
from google.colab import files
uploaded = files.upload()

Saving train.zip to train.zip


In [9]:
import os

# Define paths for data
raw_data_dir = "/content/data/raw/"
processed_data_dir = "/content/data/processed/"

# Ensure directories exist
os.makedirs(raw_data_dir, exist_ok=True)
os.makedirs(processed_data_dir, exist_ok=True)


In [13]:
import zipfile

# Define function to extract zip files
def extract_zip(file_path, output_dir, password):
    try:
        with zipfile.ZipFile(file_path, 'r') as zf:
            zf.extractall(path=output_dir, pwd=password.encode('utf-8'))
        print(f"Extracted: {file_path} to {output_dir}")
    except Exception as e:
        print(f"Error extracting {file_path}: {e}")
        # Define paths to uploaded zip files
batch_zip_path = 'train.zip'  # Update the name to the correct file name
dev_zip_path = 'dev.zip'      # Update the name to the correct file name
gold_labels_zip_path = 'gold_labels.zip'  # Update the name to the correct file name

# Extract files
extract_zip(batch_zip_path, raw_data_dir, "narratives5202trainTHREE")
extract_zip(dev_zip_path, raw_data_dir, "devNarratives5202")
extract_zip(gold_labels_zip_path, raw_data_dir, "narratives5202devCLEANED")

print("All files extracted successfully!")





Extracted: train.zip to /content/data/raw/
Extracted: dev.zip to /content/data/raw/
Extracted: gold_labels.zip to /content/data/raw/
All files extracted successfully!


In [14]:
import os

# List files in the raw data directory
print("Files in raw_data_dir:", os.listdir(raw_data_dir))


Files in raw_data_dir: ['RU', 'BG', 'HI', 'PT', 'EN', 'dev-documents_4_December']


In [9]:
import zipfile

# Upload your zip files
from google.colab import files
uploaded = files.upload()

# Extract files
def extract_zip(zip_path, output_dir, password=None):
    with zipfile.ZipFile(zip_path, 'r') as zf:
        zf.extractall(output_dir, pwd=password.encode() if password else None)

# Extract training and dev sets
extract_zip("train.zip", "./data/train/", password="narratives5202trainTHREE")
extract_zip("dev.zip", "./data/dev/", password="devNarratives5202")
extract_zip("gold_labels.zip", "./data/gold_labels/", password="narratives5202devCLEANED")

print("Files extracted!")


Saving dev.zip to dev.zip
Saving gold_labels.zip to gold_labels.zip
Saving train.zip to train.zip
Files extracted!


In [15]:
# List files in the 'EN' folder to check the contents of the English data
english_files = os.listdir('/content/data/raw/EN')
print("English files:", english_files)


English files: ['subtask-2-documents', 'subtask-1-entity-mentions.txt', 'subtask-3-annotations.txt', 'subtask-1-documents', 'subtask-3-dominant-narratives.txt', 'subtask-2-annotations.txt', 'subtask-3-documents', 'raw-documents', 'subtask-1-annotations.txt']


In [16]:
# List files in the 'EN' folder to check the contents of the English data
Hindi_files = os.listdir('/content/data/raw/HI')
print("hINDI files:", Hindi_files)

hINDI files: ['subtask-2-documents', 'subtask-1-entity-mentions.txt', 'subtask-3-annotations.txt', 'subtask-1-documents', 'subtask-3-dominant-narratives.txt', 'subtask-2-annotations.txt', 'subtask-3-documents', 'raw-documents', 'subtask-1-annotations.txt']


In [11]:
# List files in the extracted folder
import os

# List files in the training data directory
train_directory = "./data/train/"
print("Files in train directory:", os.listdir(train_directory))


Files in train directory: ['PT', 'RU', 'BG', 'EN', 'HI']


In [19]:
import os

# List contents of the directory
english_documents_dir = '/content/data/raw/EN/subtask-1-documents'
english_documents_files = os.listdir(english_documents_dir)

# Display the files in the directory
english_documents_files


['EN_UA_DEV_100029.txt',
 'EN_CC_200035.txt',
 'EN_UA_DEV_100013.txt',
 'EN_CC_200033.txt',
 'EN_CC_200070.txt',
 'EN_UA_DEV_100019.txt',
 'EN_CC_200046.txt',
 'EN_CC_200077.txt',
 'EN_UA_DEV_100026.txt',
 'EN_UA_DEV_214.txt',
 'EN_CC_200036.txt',
 'EN_UA_DEV_213.txt',
 'EN_UA_DEV_100033.txt',
 'EN_CC_200034.txt',
 'EN_CC_200071.txt',
 'EN_CC_200069.txt',
 'EN_UA_DEV_100012.txt',
 'EN_CC_200064.txt',
 'EN_CC_200081.txt',
 'EN_CC_200079.txt',
 'EN_CC_200040.txt',
 'EN_UA_DEV_20.txt',
 'EN_UA_DEV_100002.txt',
 'EN_CC_200054.txt',
 'EN_UA_DEV_215.txt',
 'EN_CC_200053.txt',
 'EN_CC_200047.txt']

Load English Documents

In [21]:
# Define the directory containing the English text files
english_documents_dir = '/content/data/raw/EN/subtask-1-documents'

# List all English text files
english_documents_files = os.listdir(english_documents_dir)

# Initialize an empty list to store the content of all files
english_documents = []

# Read each file and append its content to the list
for filename in english_documents_files:
    file_path = os.path.join(english_documents_dir, filename)
    with open(file_path, 'r') as file:
        content = file.readlines()
        english_documents.append(content)

# Check the first few lines of the first document to ensure successful loading
english_documents[0][:10]  # Show the first 10 lines of the first document


['General Milley: Russian military stocks rapidly depleting, soldiers demoralized \n',
 '\n',
 '“Russia remains isolated. Their military stocks are rapidly depleting, the soldiers are demoralized, untrained, unmotivated, conscripts, and convicts, and their leadership is failing them, having already failed in their strategic objectives,” Milley said during a press conference at Pentagon following the Ukraine Defense Contact Group meeting, an Ukrinform correspondent reports.\n',
 '\n',
 'Russia is increasingly relying on other countries such as Iran and North Korea, he noted.\n',
 '\n',
 'At the same time, Russian soldiers are thrown into the chaos of war wave after wave. “The severely undertrained, poorly led, poorly equipped Russian forces are conducting mindless frontal attacks and sacrificing hundreds per day… It’s obvious to the world and it should be obvious to Putin that these objectives are no longer achievable by continuing this war,” Milley believes.\n',
 '\n',
 'Commenting on 

In [22]:
# Define the directory containing the Hindi text files
hindi_documents_dir = '/content/data/raw/HI/subtask-1-documents'

# List all Hindi text files
hindi_documents_files = os.listdir(hindi_documents_dir)

# Initialize an empty list to store the content of all files
hindi_documents = []

# Read each file and append its content to the list
for filename in hindi_documents_files:
    file_path = os.path.join(hindi_documents_dir, filename)
    with open(file_path, 'r') as file:
        content = file.readlines()
        hindi_documents.append(content)

# Check the first few lines of the first document to ensure successful loading
hindi_documents[0][:10]  # Show the first 10 lines of the first document


['रूस-यूक्रेन, इजरायल-ईरान... क्या ये देश शुरू कर चुके हैं तीसरा World War?\n',
 '\n',
 'रूस ने साफ-साफ कह दिया है कि अगर नाटो यूक्रेन के समर्थन में उसके शहरों पर हमला करता है तो परमाणु युद्ध टाला नहीं जा सकता. रूस ने कहा कि यूक्रेन के सहारे अगर नाटो रूस के शहरों पर हमला करने की हिमाकत करता है, तो ये बेवकूफी होगी. यह असंभव है. क्योंकि यूक्रेन के पास लंबी दूरी का हमला करने की तकनीक और विज्ञान नहीं है. लेकिन नाटो कर सकता है.\n',
 'रूस ने इस बीच दोनेत्सक इलाके में काफी बार सफलता हासिल की है. रक्षा एक्सपर्ट्स का मानना है कि अगर यूक्रेन इसी तरह से हमला करता रहा तो अगले साल मार्च-अप्रैल तक ये बेहद कमजोर हो जाएगा. इसलिए कुर्स्क इलाके में नाटो के सपोर्ट से हमला किया गया. रूस के राष्ट्रपति व्लादिमीर पुतिन ने स्पष्ट कहा है कि अगर नाटो ने सीधा हमला किया तो जंग रूस और नाटो की होगी.\n',
 'यह भी पढ़ें: पहला Tejas MK1-A फाइटर जेट अक्टूबर अंत तक मिलेगा वायुसेना को, ये है खासियत\n',
 'अगर नाटो-यूक्रेन हमला करते हैं तो रूस क्या करेगा? तो रूस के पास क्या-क्या विकल्प हैं.\n',
 '- रूस मार्शल लॉ लगाकर अपने 

Load English and Hindi Annotations

In [23]:
# Load English annotations
english_annotations_file = '/content/data/raw/EN/subtask-1-annotations.txt'
with open(english_annotations_file, 'r') as file:
    english_annotations = file.readlines()

# Load Hindi annotations (same logic for Hindi annotations)
hindi_annotations_file = '/content/data/raw/HI/subtask-1-annotations.txt'
with open(hindi_annotations_file, 'r') as file:
    hindi_annotations = file.readlines()

# Check first few lines of annotations to ensure proper loading
english_annotations[:5], hindi_annotations[:5]


(['EN_UA_DEV_100012.txt\tWashington\t1441\t1450\tAntagonist\tCorrupt\n',
  'EN_UA_DEV_100012.txt\tAmericans\t2397\t2405\tAntagonist\tCorrupt\n',
  'EN_CC_200053.txt\tRishi Sunak\t85\t95\tAntagonist\tDeceiver\n',
  'EN_CC_200040.txt\tExtinction Rebellion\t189\t208\tAntagonist\tTerrorist\n',
  'EN_CC_200070.txt\tEric Garcetti\t89\t101\tAntagonist\tDeceiver\tCorrupt\n'],
 ['HI_26.txt\tमोदी\t1123\t1126\tProtagonist\tGuardian\n',
  'HI_26.txt\tभारत\t3085\t3088\tProtagonist\tPeacemaker\n',
  'HI_26.txt\tराष्ट्रपति वोलोदोमीर\t3540\t3559\tAntagonist\tForeign Adversary\n',
  'HI_26.txt\tरूस\t3614\t3616\tAntagonist\tInstigator\n',
  'HI_26.txt\tकीव\t3621\t3623\tInnocent\tVictim\n'])

Merging Annotations with Documents

In [24]:
# Example: Merging English documents and their annotations by filename (adjust logic if needed)

# Assuming annotations are in a format like "document_id annotation", you can split them
english_merged_data = []
for doc_file, annotation in zip(english_documents_files, english_annotations):
    doc_id = doc_file.split('.')[0]  # Assuming the ID is part of the filename
    doc_content = english_documents_files[english_documents_files.index(doc_file)]
    english_merged_data.append({'id': doc_id, 'content': doc_content, 'annotation': annotation.strip()})

# Check merged English data
english_merged_data[:5]

# Repeat for Hindi data
hindi_merged_data = []
for doc_file, annotation in zip(hindi_documents_files, hindi_annotations):
    doc_id = doc_file.split('.')[0]  # Assuming the ID is part of the filename
    doc_content = hindi_documents_files[hindi_documents_files.index(doc_file)]
    hindi_merged_data.append({'id': doc_id, 'content': doc_content, 'annotation': annotation.strip()})

# Check merged Hindi data
hindi_merged_data[:5]


[{'id': 'HI_95',
  'content': 'HI_95.txt',
  'annotation': 'HI_26.txt\tमोदी\t1123\t1126\tProtagonist\tGuardian'},
 {'id': 'HI_19',
  'content': 'HI_19.txt',
  'annotation': 'HI_26.txt\tभारत\t3085\t3088\tProtagonist\tPeacemaker'},
 {'id': 'HI_159',
  'content': 'HI_159.txt',
  'annotation': 'HI_26.txt\tराष्ट्रपति वोलोदोमीर\t3540\t3559\tAntagonist\tForeign Adversary'},
 {'id': 'HI_134',
  'content': 'HI_134.txt',
  'annotation': 'HI_26.txt\tरूस\t3614\t3616\tAntagonist\tInstigator'},
 {'id': 'HI_34',
  'content': 'HI_34.txt',
  'annotation': 'HI_26.txt\tकीव\t3621\t3623\tInnocent\tVictim'}]

Preparing Training and Test Data

In [25]:
from sklearn.model_selection import train_test_split

# Split English data into training and testing sets
english_train_data, english_test_data = train_test_split(english_merged_data, test_size=0.2, random_state=42)

# Split Hindi data into training and testing sets
hindi_train_data, hindi_test_data = train_test_split(hindi_merged_data, test_size=0.2, random_state=42)

# Check size of train and test data
len(english_train_data), len(english_test_data), len(hindi_train_data), len(hindi_test_data)


(21, 6, 28, 7)

Preprocessing (Tokenization and Formatting)

In [26]:
from transformers import AutoTokenizer

# Load a pre-trained tokenizer
tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")

# Tokenize English training data
english_tokenized_data = []
for item in english_train_data:
    encoded = tokenizer(item['content'], padding='max_length', truncation=True, max_length=512)
    english_tokenized_data.append({'id': item['id'], 'input_ids': encoded['input_ids'], 'attention_mask': encoded['attention_mask'], 'labels': item['annotation']})

# Tokenize Hindi training data
hindi_tokenized_data = []
for item in hindi_train_data:
    encoded = tokenizer(item['content'], padding='max_length', truncation=True, max_length=512)
    hindi_tokenized_data.append({'id': item['id'], 'input_ids': encoded['input_ids'], 'attention_mask': encoded['attention_mask'], 'labels': item['annotation']})

# Check tokenized data for English and Hindi
english_tokenized_data[:2], hindi_tokenized_data[:2]


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

([{'id': 'EN_UA_DEV_100012',
   'input_ids': [101,
    4372,
    1035,
    25423,
    1035,
    16475,
    1035,
    6694,
    12521,
    1012,
    19067,
    2102,
    102,
    0,
    0,
    0,
    0,
    0,
    0,
    0,
    0,
    0,
    0,
    0,
    0,
    0,
    0,
    0,
    0,
    0,
    0,
    0,
    0,
    0,
    0,
    0,
    0,
    0,
    0,
    0,
    0,
    0,
    0,
    0,
    0,
    0,
    0,
    0,
    0,
    0,
    0,
    0,
    0,
    0,
    0,
    0,
    0,
    0,
    0,
    0,
    0,
    0,
    0,
    0,
    0,
    0,
    0,
    0,
    0,
    0,
    0,
    0,
    0,
    0,
    0,
    0,
    0,
    0,
    0,
    0,
    0,
    0,
    0,
    0,
    0,
    0,
    0,
    0,
    0,
    0,
    0,
    0,
    0,
    0,
    0,
    0,
    0,
    0,
    0,
    0,
    0,
    0,
    0,
    0,
    0,
    0,
    0,
    0,
    0,
    0,
    0,
    0,
    0,
    0,
    0,
    0,
    0,
    0,
    0,
    0,
    0,
    0,
    0,
    0,
    0,
    0,
    0,
    0,
    0,
    0,
    0,


 Model Training Framework (from Hugging Face

In [27]:
from transformers import AutoModelForSequenceClassification, Trainer, TrainingArguments

# Load a pre-trained multilingual model (mBERT or XLM-R)
model = AutoModelForSequenceClassification.from_pretrained("bert-base-multilingual-cased", num_labels=2)

# Define training arguments
training_args = TrainingArguments(
    output_dir="/content/output",          # Output directory
    num_train_epochs=3,                    # Number of training epochs
    per_device_train_batch_size=8,         # Batch size for training
    per_device_eval_batch_size=8,          # Batch size for evaluation
    warmup_steps=500,                      # Number of warmup steps
    weight_decay=0.01,                     # Strength of weight decay
    logging_dir="/content/logs",           # Directory for logging
    logging_steps=10,
    evaluation_strategy="epoch",           # Evaluate after each epoch
)

# Define the Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=english_tokenized_data,   # Training dataset (use the tokenized English data)
    eval_dataset=english_tokenized_data,    # Evaluation dataset (could be the test set)
)

# Train the model
trainer.train()


config.json:   0%|          | 0.00/625 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/714M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-multilingual-cased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1575: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.
wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:


Abort: 

In [32]:
import time
from transformers import AutoModelForSequenceClassification, Trainer, TrainingArguments

# Simulated verification with an API key
def simulate_verification_api(api_key):
    # Simulated API response
    if api_key == "your_api_key_here":
        return {"status": "success", "message": "Verification successful!"}
    else:
        return {"status": "failed", "message": "Verification error occurred."}

# Fine-tuning function
def verify_and_train_with_retry(api_key, retries=3, wait_time=2):
    for attempt in range(retries):
        print(f"Verification attempt {attempt + 1}/{retries}...")
        verification_response = simulate_verification_api(api_key)

        if verification_response["status"] == "success":
            print("Verification successful! Proceeding with training...")

            # Load pre-trained multilingual model (mBERT or XLM-R)
            model = AutoModelForSequenceClassification.from_pretrained("bert-base-multilingual-cased", num_labels=2)

            # Define training arguments
            training_args = TrainingArguments(
                output_dir="/content/output",
                num_train_epochs=3,
                per_device_train_batch_size=8,
                per_device_eval_batch_size=8,
                warmup_steps=500,
                weight_decay=0.01,
                logging_dir="/content/logs",
                logging_steps=10,
                evaluation_strategy="epoch",
            )

            # Define Trainer
            trainer = Trainer(
                model=model,
                args=training_args,
                train_dataset=english_tokenized_data,  # Replace with your actual training dataset
                eval_dataset=english_tokenized_data,  # Replace with your actual validation dataset
            )

            # Start training
            trainer.train()
            print("Training completed.")
            return
        else:
            print(f"Verification failed: {verification_response['message']}")
            if attempt < retries - 1:
                print(f"Retrying in {wait_time} seconds...")
                time.sleep(wait_time)  # Wait before retrying
            else:
                print("Maximum retries reached. Aborting training.")
                return

# Example call with retry logic
api_key = "your_api_key_here"  # Replace with your actual API key
verify_and_train_with_retry(api_key)


Verification attempt 1/3...
Verification successful! Proceeding with training...


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-multilingual-cased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:


Abort: 

In [34]:
import os

# Disable wandb logging
os.environ["WANDB_DISABLED"] = "true"


In [38]:
!pip install datasets


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 17.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 8.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 10.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 8.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 14.6 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.


In [39]:
from datasets import Dataset
from transformers import AutoTokenizer, AutoModelForSequenceClassification, Trainer, TrainingArguments

# Load the tokenizer for pre-processing
tokenizer = AutoTokenizer.from_pretrained("bert-base-multilingual-cased")

# Sample dataset creation (replace with your actual dataset)
# Ensure your dataset contains 'text' and 'label' columns
data = {"text": ["This is a positive example.", "This is a negative example."],
        "label": [1, 0]}  # 1 for positive, 0 for negative labels

# Create Dataset
dataset = Dataset.from_dict(data)

# Tokenize the dataset
def tokenize_function(examples):
    return tokenizer(examples['text'], padding="max_length", truncation=True)

tokenized_datasets = dataset.map(tokenize_function, batched=True)

# Prepare the tokenized datasets for training and validation
train_dataset = tokenized_datasets
eval_dataset = tokenized_datasets

# Load pre-trained multilingual model (mBERT or XLM-R)
model = AutoModelForSequenceClassification.from_pretrained("bert-base-multilingual-cased", num_labels=2)

# Define training arguments
training_args = TrainingArguments(
    output_dir="/content/output",
    num_train_epochs=3,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    warmup_steps=500,
    weight_decay=0.01,
    logging_dir="/content/logs",
    logging_steps=10,
    evaluation_strategy="epoch",
    disable_tqdm=True,  # Disable tqdm for logging
)

# Define Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,  # Ensure the dataset contains the 'labels'
    eval_dataset=eval_dataset,  # Ensure the dataset contains the 'labels'
)

# Start training
trainer.train()
print("Training completed.")


tokenizer_config.json:   0%|          | 0.00/49.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/996k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.96M [00:00<?, ?B/s]

Map:   0%|          | 0/2 [00:00<?, ? examples/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-multilingual-cased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).


{'eval_loss': 0.6861424446105957, 'eval_runtime': 9.5325, 'eval_samples_per_second': 0.21, 'eval_steps_per_second': 0.105, 'epoch': 1.0}
{'eval_loss': 0.6860710382461548, 'eval_runtime': 3.3831, 'eval_samples_per_second': 0.591, 'eval_steps_per_second': 0.296, 'epoch': 2.0}
{'eval_loss': 0.68593430519104, 'eval_runtime': 3.7426, 'eval_samples_per_second': 0.534, 'eval_steps_per_second': 0.267, 'epoch': 3.0}
{'train_runtime': 97.0182, 'train_samples_per_second': 0.062, 'train_steps_per_second': 0.031, 'train_loss': 0.7011544704437256, 'epoch': 3.0}
Training completed.


In [40]:
# Save the trained model
model.save_pretrained("/content/output/model")

# Save the tokenizer
tokenizer.save_pretrained("/content/output/tokenizer")


('/content/output/tokenizer/tokenizer_config.json',
 '/content/output/tokenizer/special_tokens_map.json',
 '/content/output/tokenizer/vocab.txt',
 '/content/output/tokenizer/added_tokens.json',
 '/content/output/tokenizer/tokenizer.json')

In [41]:
from transformers import AutoModelForSequenceClassification, AutoTokenizer

# Load the model and tokenizer
model = AutoModelForSequenceClassification.from_pretrained("/content/output/model")
tokenizer = AutoTokenizer.from_pretrained("/content/output/tokenizer")

# Use the model for predictions
inputs = tokenizer("Your input text here", return_tensors="pt", truncation=True, padding=True)
outputs = model(**inputs)

# Get the predicted class (assuming binary classification)
predicted_class = outputs.logits.argmax(dim=-1).item()
print(f"Predicted class: {predicted_class}")


Predicted class: 1


In [43]:
from datasets import load_dataset

# Example of loading a dataset from Hugging Face
test_dataset = load_dataset("glue", "mrpc", split="test")


README.md:   0%|          | 0.00/35.3k [00:00<?, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/649k [00:00<?, ?B/s]

validation-00000-of-00001.parquet:   0%|          | 0.00/75.7k [00:00<?, ?B/s]

test-00000-of-00001.parquet:   0%|          | 0.00/308k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/3668 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/408 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/1725 [00:00<?, ? examples/s]

In [50]:
print(test_dataset[:5])  # Display the first 5 entries


{'sentence1': ["PCCW 's chief operating officer , Mike Butcher , and Alex Arena , the chief financial officer , will report directly to Mr So .", "The world 's two largest automakers said their U.S. sales declined more than predicted last month as a late summer sales frenzy caused more of an industry backlash than expected .", 'According to the federal Centers for Disease Control and Prevention ( news - web sites ) , there were 19 reported cases of measles in the United States in 2002 .', 'A tropical storm rapidly developed in the Gulf of Mexico Sunday and was expected to hit somewhere along the Texas or Louisiana coasts by Monday night .', "The company didn 't detail the costs of the replacement and repairs ."], 'sentence2': ['Current Chief Operating Officer Mike Butcher and Group Chief Financial Officer Alex Arena will report to So .', 'Domestic sales at both GM and No. 2 Ford Motor Co. declined more than predicted as a late summer sales frenzy prompted a larger-than-expected industr

In [58]:
from datasets import Dataset

# Example: Creating a dataset manually
test_data = {
    "sentence1": ["Sentence 1", "Sentence 2", "Sentence 3"],
    "label": [0, 1, 0]
}

test_dataset = Dataset.from_dict(test_data)


In [53]:
def tokenize_function(examples):
    return tokenizer(examples["sentence1"], padding="max_length", truncation=True)


In [60]:
test_dataset = test_dataset.map(tokenize_function, batched=True)


Map:   0%|          | 0/3 [00:00<?, ? examples/s]

In [61]:
evaluation_results = trainer.evaluate(test_dataset)
print(f"Test Loss: {evaluation_results['eval_loss']}")


{'eval_loss': 0.714738130569458, 'eval_runtime': 11.8322, 'eval_samples_per_second': 0.254, 'eval_steps_per_second': 0.085, 'epoch': 3.0}
Test Loss: 0.714738130569458


In [12]:
# Read only English articles
def read_english_articles(base_folder):
    english_folder = os.path.join(base_folder, 'EN')  # Specify the English folder
    articles = []
    if os.path.isdir(english_folder):
        print(f"Processing folder: EN")
        for file in os.listdir(english_folder):
            if file.endswith(".txt"):
                file_path = os.path.join(english_folder, file)
                with open(file_path, 'r', encoding='utf-8') as f:
                    content = f.read().strip().split("\n")
                    print(f"Reading file: {file_path}, Content length: {len(content)}")
                    title = content[0] if len(content) > 0 else ""
                    body = " ".join(content[2:])  # Skip blank line after title
                    articles.append({"id": file, "title": title, "body": body})
    return pd.DataFrame(articles)

# Load only English training data
train_data_english = read_english_articles(train_directory)
print("English Training Data Sample:\n", train_data_english.head())


Processing folder: EN
Reading file: ./data/train/EN/subtask-3-annotations.txt, Content length: 203
Reading file: ./data/train/EN/subtask-2-annotations.txt, Content length: 399
Reading file: ./data/train/EN/subtask-1-annotations.txt, Content length: 686
English Training Data Sample:
                           id  \
0  subtask-3-annotations.txt   
1  subtask-2-annotations.txt   
2  subtask-1-annotations.txt   

                                               title  \
0  EN_CC_100013.txt\tCC: Criticism of climate mov...   
1  EN_CC_100013.txt\tCC: Criticism of climate mov...   
2  EN_CC_100013.txt\tBill Gates\t93\t102\tAntagon...   

                                                body  
0  EN_CC_100005.txt\tCC: Criticism of climate mov...  
1  EN_UA_300017.txt\tOther\tOther EN_CC_100021.tx...  
2  EN_CC_100013.txt\tJeffrey Epstein\t2005\t2019\...  


Handling Sub-narrative Labels:

In [3]:
!pip install transformers datasets torch


In [2]:
!pip install evaluate


In [6]:
import torch
from datasets import load_dataset
from transformers import AutoTokenizer, AutoModelForSequenceClassification, Trainer, TrainingArguments
import evaluate  # For metrics

# Load the accuracy metric
accuracy_metric = evaluate.load("accuracy")

# Example of how you might compute accuracy
def compute_metrics(p):
    predictions, labels = p
    predictions = predictions.argmax(axis=-1)
    return accuracy_metric.compute(predictions=predictions, references=labels)


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


In [7]:
# Load the dataset from a local path or online URL
dataset = load_dataset('imdb')


# Inspect the dataset structure
print(dataset)


DatasetDict({
    train: Dataset({
        features: ['text', 'label'],
        num_rows: 25000
    })
    test: Dataset({
        features: ['text', 'label'],
        num_rows: 25000
    })
    unsupervised: Dataset({
        features: ['text', 'label'],
        num_rows: 50000
    })
})


Preprocess the Data (Tokenization)

In [8]:
from transformers import AutoTokenizer

# Load tokenizer
tokenizer = AutoTokenizer.from_pretrained('bert-base-uncased')

# Tokenize the dataset
def tokenize_function(examples):
    return tokenizer(examples['text'], padding="max_length", truncation=True)

# Apply the tokenization function to the dataset
tokenized_datasets = dataset.map(tokenize_function, batched=True)


Map:   0%|          | 0/50000 [00:00<?, ? examples/s]

In [9]:
train_dataset = tokenized_datasets['train']
test_dataset = tokenized_datasets['test']


Create Data Collator

In [10]:
!pip install --upgrade transformers


In [11]:
from transformers import DataCollatorWithPadding

data_collator = DataCollatorWithPadding(tokenizer)


 Model Setup and Fine-Tuning

In [6]:
import os
os.environ["WANDB_DISABLED"] = "true"


In [7]:
from transformers import TrainingArguments

training_args = TrainingArguments(
    output_dir="./results",
    num_train_epochs=3,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    report_to=None  # This will disable W&B and other integrations
)


Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).


In [18]:
from transformers import AutoModelForSequenceClassification, AutoTokenizer

# Load the pre-trained model
model_name = "bert-base-uncased"
model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=2)  # Adjust num_labels as per your task

# Load the tokenizer
tokenizer = AutoTokenizer.from_pretrained(model_name)


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [2]:
from datasets import load_dataset

# Example of loading a dataset from Hugging Face
train_dataset = load_dataset('glue', 'mrpc', split='train')
eval_dataset = load_dataset('glue', 'mrpc', split='validation')


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


In [3]:
from transformers import AutoTokenizer

# Load the tokenizer for the pre-trained model (e.g., BERT)
tokenizer = AutoTokenizer.from_pretrained('bert-base-uncased')

# Tokenize the datasets
def tokenize_function(examples):
    return tokenizer(examples['sentence1'], examples['sentence2'], padding="max_length", truncation=True)

# Apply the tokenizer to both training and evaluation datasets
train_dataset = train_dataset.map(tokenize_function, batched=True)
eval_dataset = eval_dataset.map(tokenize_function, batched=True)

# Set the format for PyTorch (optional if you are using PyTorch)
train_dataset.set_format(type='torch', columns=['input_ids', 'attention_mask', 'label'])
eval_dataset.set_format(type='torch', columns=['input_ids', 'attention_mask', 'label'])


In [4]:
from transformers import AutoModelForSequenceClassification

# Load the model for sequence classification
model = AutoModelForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=2)


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [14]:
from transformers import TrainingArguments

training_args = TrainingArguments(
    output_dir='./results',
    evaluation_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=8,  # Decrease batch size
    per_device_eval_batch_size=8,
    num_train_epochs=3,
    weight_decay=0.01,
    logging_dir='./logs',
    report_to='none',
    fp16=True,  # Enable mixed precision training
)


In [18]:
from transformers import Trainer, TrainingArguments

# Create the Trainer object
trainer = Trainer(
    model=model,                         # The model to train
    args=training_args,                   # Training arguments
    train_dataset=train_dataset,          # The training dataset
    eval_dataset=eval_dataset,            # The evaluation dataset
    tokenizer=tokenizer,                  # Tokenizer for processing inputs
)


<ipython-input-18-774d5126826a>:4: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


In [19]:
model = AutoModelForSequenceClassification.from_pretrained('distilbert-base-uncased', num_labels=2)


Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [20]:
def tokenize_function(examples):
    return tokenizer(examples['sentence1'], examples['sentence2'], padding="max_length", truncation=True, max_length=128)  # Reduce max_length

train_dataset = train_dataset.map(tokenize_function, batched=True)
eval_dataset = eval_dataset.map(tokenize_function, batched=True)


Map:   0%|          | 0/3668 [00:00<?, ? examples/s]

Map:   0%|          | 0/408 [00:00<?, ? examples/s]

In [ ]:
# Start training
trainer.train()


Epoch,Training Loss,Validation Loss


Epoch,Training Loss,Validation Loss
1,No log,0.369839
2,0.508600,0.401734


In [ ]:
# Evaluate the model
eval_results = trainer.evaluate()

# Print evaluation results
print(eval_results)
